In [1]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 23.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [2]:
import torch
from torch import nn
import os
from torchtext.data.utils import get_tokenizer
import spacy

/home/jakub/anaconda3/envs/transformers/lib/python3.12/site-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [3]:
with open("./data/pol.txt") as f:
    lines = f.readlines()
for i, line in enumerate(lines):
    lines[i] = line.split('\t')


In [5]:
pl_tokenizer = spacy.load("pl_core_news_sm")
en_tokenizer = spacy.load("en_core_web_sm")

In [7]:
tokens = pl_tokenizer("Dzisiaj mam dobry dzien")

In [22]:
en_list = [line[0] for line in lines]
pl_list = [line[1] for line in lines]

In [24]:
pl_list

['Idź.',
 'Cześć.',
 'Uciekaj!',
 'Biegnij.',
 'Uciekaj.',
 'Kto?',
 'O, dziamdzia zaprzała jej szadź!',
 'Łał!',
 'Unik!',
 'Pali się!',
 'Strzelaj!',
 'Ognia!',
 'Pomocy!',
 'Schowaj się.',
 'Skacz!',
 'Skok.',
 'Zostań.',
 'Stój!',
 'Zatrzymaj się!',
 'Czekaj!',
 'Zaczekaj!',
 'Poczekaj!',
 'Czekajcie!',
 'Poczekajcie!',
 'Zaczekajcie!',
 'Niech pan zaczeka!',
 'Niech pani zaczeka!',
 'Czekajcie.',
 'Zaczekaj.',
 'Czekaj.',
 'Poczekaj.',
 'Poczekajcie.',
 'Zaczekajcie.',
 'Zaczynaj.',
 'Zaczynajcie.',
 'Zrób to.',
 'Cześć.',
 'Cześć.',
 'Pośpiesz się!',
 'Ukryłem się.',
 'Pobiegłem.',
 'Pobiegłam.',
 'Rozumiem.',
 'Widzę.',
 'Próbuje.',
 'Wygrałem!',
 'Wygrałem.',
 'Zwyciężyłem.',
 'O nie!',
 'Wyluzuj.',
 'Strzelaj!',
 'Ognia!',
 'Uśmiech.',
 'Przepraszam?',
 'Spytaj mnie.',
 'Atak!',
 'Do ataku!',
 'Kup je.',
 'Kup go.',
 'Kup ją.',
 'Na zdrowie!',
 'Twoje zdrowie!',
 'Zjedz to.',
 'Wydech.',
 'Stój!',
 'Nie ruszaj się!',
 'Wstawaj.',
 'Idź już.',
 'Rozumiem!',
 'Rozumiesz?',
 'Kap

In [ ]:
ENCODER_EMBEDDING_DIM = 256
DECODER_EMBEDDING_DIM = 256
ENCODER_HIDDEN_DIM = 256
DECODER_HIDDEN_DIM = 256
SAVE_DIR = os.path.join(".", "models")

In [ ]:
class BahdanauSeq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder.to(device)
        self.decoder = decoder.to(device)
        self.device = device

    def forward(self, x):
        outputs, hidden = self.encoder(x)


class BahdanauEncoder(nn.Module):
    def __init__(self, input_dim, embedding_dim,
                 encoder_hidden_dim, decoder_hidden_dim, dropout_p):
        super().__init__()
        self.input_dim = input_dim
        self.embedding_dim = embedding_dim
        self.encoder_hidden_dim = encoder_hidden_dim
        self.decoder_hidden_dim = decoder_hidden_dim
        self.dropout_p = dropout_p

        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.gru = nn.GRU(embedding_dim, encoder_hidden_dim,
                          bidirectional=True)
        # Times 2 because of bidirectional (I guess)
        self.linear = nn.Linear(encoder_hidden_dim*2, decoder_hidden_dim)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        outputs, hidden = self.gru(embedded)
        hidden = torch.tanh(self.linear(
            torch.cat((hidden[-2, :, :], hidden[-1, :, :]),
                      dim=1)
        ))
        return outputs, hidden


class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size, query_size=None,
                 key_size=None, dropout_p=0.15):
        super().__init__()
        self.hidden_size = hidden_size
        self.query_size = hidden_size if query_size is None else query_size
        # Assume that encoder is bidirectional (times 2)
        self.key_size = 2*hidden_size if key_size is None else key_size

        self.query_layer = nn.Linear(self.query_size, hidden_size)
        self.key_layer = nn.Linear(self.key_size, hidden_size)
        self.energy_layer = nn.Linear(hidden_size, 1)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, hidden, encoder_outputs, src_mask=None):
        query_out = self.query_layer(hidden)
        key_out = self.key_layer(encoder_outputs)
        energy_input = torch.tanh(query_out + key_out)
        energies = self.energy_layer(energy_input).squeeze(2)

        if src_mask is not None:
            energies.data.masked_fill_(src_mask == 0, float("-inf"))

        weights = torch.softmax(energies, dim=0)
        return weights.transpose(0, 1)


class BahdanauDecoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, encoder_hidden_dim,
                 decoder_hidden_dim, attention, dropout_p):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.encoder_hidden_dim = encoder_hidden_dim
        self.decoder_hidden_dim = decoder_hidden_dim
        self.dropout_p = dropout_p
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.attention = attention
        self.gru = nn.GRU((encoder_hidden_dim*2) +
                          embedding_dim, decoder_hidden_dim)
        self.out = nn.Linear((encoder_hidden_dim*2) +
                             embedding_dim+decoder_hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input, hidden, encoder_outputs, src_mask=None):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        attentions = self.attention(hidden, encoder_outputs, src_mask)
        a = attentions.unsqueeze(1)
        encoder_outputs = encoder_outputs.transpose(0, 1)

        '''Multiplying two matrices that are batched. If 1st batched matrix size is (10, 3, 5) 
        and 2nd batched matrix size is (10, 5, 6) then output matrix size will be (10, 3, 6)'''
        weighted = torch.bmm(a, encoder_outputs)
        weighted = weighted.transpose(0, 1)

        rnn_input = torch.cat((embedded, weighted), dim=2)
        output, hidden = self.gru(rnn_input, hidden.unsqueeze(0))

        assert (output == hidden).all()

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)

        linear_input = torch.cat((output, weighted, embedded), dim=1)

        output = self.out(linear_input)
        return output, hidden.squeeze(0), attentions


In [ ]:
def main():

    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
    enc = BahdanauEncoder(len(en_vocab), ENCODER_EMBEDDING_DIM,
                          ENCODER_HIDDEN_DIM, DECODER_HIDDEN_DIM, 0.15)
    attn = BahdanauAttention(DECODER_HIDDEN_DIM)
    dec = BahdanauDecoder(len(fr_vocab), DECODER_EMBEDDING_DIM,
                          ENCODER_HIDDEN_DIM, DECODER_HIDDEN_DIM, attn, 0.15)
    seq2seq = BahdanauSeq2Seq(enc, dec, device)


if __name__ == '__main__':
    main()